# M5 Forecasting - Previsão de Demanda Walmart

## 🎯 Objetivo
Desenvolver um modelo preditivo de demanda para itens de varejo usando a base de dados Walmart M5 Forecasting.

## 📋 Estrutura do Projeto
1. **Carregamento e Análise Exploratória**
2. **Engenharia de Features**
3. **Modelagem (LightGBM/XGBoost)**
4. **Avaliação e Visualizações**
5. **Previsões Finais**

In [ ]:
# Importações básicas
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Adiciona o diretório src ao path
project_root = r"c:\Users\thiago.santos\Desktop\PESSOAL\RandomF_XGB\m5_project"
sys.path.append(os.path.join(project_root, 'src'))

# Importa módulos do projeto
from data_loader import M5DataLoader
from feature_engineering import M5FeatureEngineer
from modeling import M5Model, M5Ensemble
from visualization import M5Visualizer
from utils import MemoryManager, reduce_memory_usage, timer, check_environment
import config

print("✅ Módulos carregados com sucesso!")
print("\n📊 Informações do ambiente:")
env_info = check_environment()
for key, value in env_info.items():
    print(f"  {key}: {value}")

## 1. 📁 Carregamento dos Dados

In [ ]:
# Inicializa o carregador de dados
with MemoryManager(verbose=True):
    loader = M5DataLoader(config.DATA_PATH)
    
    # Carrega todos os dados
    data_dict = loader.load_all_data()
    
    # Obtém informações básicas
    basic_info = loader.get_basic_info()
    
print("\n📈 Informações Básicas dos Dados:")
for key, value in basic_info.items():
    print(f"  {key}: {value}")

print("\n💾 Uso de Memória:")
memory_info = loader.get_memory_usage()
for key, value in memory_info.items():
    print(f"  {key}: {value}")

In [ ]:
# Preprocessa o calendar
calendar_processed = loader.preprocess_calendar()

print("Calendar preprocessado!")
print(f"Shape: {calendar_processed.shape}")
print(f"Colunas adicionadas: {[col for col in calendar_processed.columns if col not in loader.calendar.columns]}")

## 2. 🔍 Análise Exploratória

In [ ]:
# Inicializa visualizador
viz = M5Visualizer()

# Para análise exploratória, vamos usar uma amostra dos dados
# Converte dados para formato long (amostra)
feature_eng = M5FeatureEngineer()

# Usa apenas uma amostra para análise exploratória
sales_sample = data_dict['sales_train'].sample(n=1000, random_state=42)
sample_melted = feature_eng.create_melted_data(sales_sample, calendar_processed)

print(f"Amostra para análise: {len(sample_melted):,} registros")

In [ ]:
# Adiciona preços à amostra
sample_with_prices = feature_eng.add_price_features(sample_melted, data_dict['sell_prices'])

# Dashboard resumo
viz.create_dashboard_summary(sample_with_prices)

In [ ]:
# Visualizações exploratórias
print("📊 Gerando visualizações exploratórias...")

# Overview das vendas
viz.plot_sales_overview(sample_with_prices)

In [ ]:
# Padrões sazonais
viz.plot_seasonal_patterns(sample_with_prices)

In [ ]:
# Análise de itens
viz.plot_item_analysis(sample_with_prices)

In [ ]:
# Análise de preços
viz.plot_price_analysis(sample_with_prices)

## 3. ⚙️ Engenharia de Features

In [ ]:
# Para o modelo completo, vamos usar mais dados
# Seleciona uma amostra maior (ajuste conforme capacidade do sistema)
sample_size = 5000  # Ajuste conforme sua memória disponível

print(f"🔧 Iniciando engenharia de features com {sample_size} itens...")

with MemoryManager(verbose=True):
    # Seleciona amostra dos dados
    sales_modeling = data_dict['sales_train'].sample(n=sample_size, random_state=42)
    
    # Aplica otimização de memória
    if config.MEMORY_OPTIMIZATION:
        sales_modeling = reduce_memory_usage(sales_modeling)
        calendar_opt = reduce_memory_usage(calendar_processed.copy())
        prices_opt = reduce_memory_usage(data_dict['sell_prices'].copy())
    else:
        calendar_opt = calendar_processed
        prices_opt = data_dict['sell_prices']
    
    # Cria features completas
    feature_data = feature_eng.create_all_features(
        sales_modeling, 
        calendar_opt, 
        prices_opt
    )
    
    print(f"\n✅ Features criadas! Shape final: {feature_data.shape}")

In [ ]:
# Lista de features para o modelo
feature_cols = feature_eng.get_feature_list(feature_data)

print(f"📝 Total de features: {len(feature_cols)}")
print(f"\nPrimeiras 20 features:")
for i, feature in enumerate(feature_cols[:20]):
    print(f"  {i+1:2d}. {feature}")
    
if len(feature_cols) > 20:
    print(f"  ... e mais {len(feature_cols) - 20} features")

In [ ]:
# Remove linhas com muitos NaN (principalmente dos lags iniciais)
print(f"📊 Dados antes da limpeza: {len(feature_data):,} registros")

# Remove linhas onde mais de 30% das features são NaN
threshold = len(feature_cols) * 0.7
feature_data_clean = feature_data.dropna(subset=feature_cols, thresh=threshold)

print(f"📊 Dados após limpeza: {len(feature_data_clean):,} registros")
print(f"📊 Registros removidos: {len(feature_data) - len(feature_data_clean):,}")

## 4. 🤖 Modelagem

In [ ]:
# Divide dados em treino e validação
from utils import split_train_validation

train_data, val_data = split_train_validation(feature_data_clean, validation_days=config.VALIDATION_DAYS)

print("\n📊 Distribuição dos dados:")
print(f"  Treino: {len(train_data):,} registros")
print(f"  Validação: {len(val_data):,} registros")

In [ ]:
# Treina modelo LightGBM
print("🚀 Treinando modelo LightGBM...")

with MemoryManager(verbose=True):
    # Inicializa modelo
    lgb_model = M5Model(model_type='lightgbm')
    
    # Prepara dados de treino
    X_train, y_train = lgb_model.prepare_training_data(train_data, feature_cols)
    
    print(f"Dados de treino preparados: {X_train.shape}")
    
    # Treina com cross-validation
    cv_results = lgb_model.time_series_split_train(X_train, y_train, n_splits=config.CV_SPLITS)
    
    print("\n✅ Treinamento concluído!")

In [ ]:
# Visualiza performance do modelo
print("📊 Performance do modelo:")

# Resumo CV
cv_summary = lgb_model.get_cv_summary()
print(cv_summary)

# Visualização
viz.plot_model_performance(cv_results)

In [ ]:
# Feature importance
print("🎯 Importância das Features:")
print(lgb_model.feature_importance.head(10))

# Visualiza feature importance
viz.plot_feature_importance(lgb_model.feature_importance, top_n=20)

## 5. 📈 Avaliação no Conjunto de Validação

In [ ]:
# Prepara dados de validação
X_val, y_val = lgb_model.prepare_training_data(val_data, feature_cols)

print(f"Dados de validação: {X_val.shape}")

# Faz predições
y_pred_val = lgb_model.predict(X_val)

# Calcula métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
val_mae = mean_absolute_error(y_val, y_pred_val)

print(f"\n📊 Métricas de Validação:")
print(f"  RMSE: {val_rmse:.4f}")
print(f"  MAE: {val_mae:.4f}")
print(f"  MAPE: {np.mean(np.abs((y_val - y_pred_val) / (y_val + 1e-8))) * 100:.2f}%")

In [ ]:
# Visualiza predições vs real
viz.plot_predictions_vs_actual(y_val.values, y_pred_val)

## 6. 🎯 Comparação com XGBoost (Opcional)

In [ ]:
# Treina modelo XGBoost para comparação
print("🚀 Treinando modelo XGBoost para comparação...")

with MemoryManager(verbose=True):
    # Inicializa modelo XGBoost
    xgb_model = M5Model(model_type='xgboost')
    
    # Treina com uma amostra menor para comparação rápida
    sample_indices = np.random.choice(len(X_train), size=min(50000, len(X_train)), replace=False)
    X_train_sample = X_train.iloc[sample_indices]
    y_train_sample = y_train.iloc[sample_indices]
    
    cv_results_xgb = xgb_model.time_series_split_train(X_train_sample, y_train_sample, n_splits=2)
    
    print("\n✅ XGBoost treinado!")

In [ ]:
# Compara modelos
y_pred_val_xgb = xgb_model.predict(X_val)

val_rmse_xgb = np.sqrt(mean_squared_error(y_val, y_pred_val_xgb))
val_mae_xgb = mean_absolute_error(y_val, y_pred_val_xgb)

print("\n🏆 Comparação de Modelos:")
print(f"LightGBM:")
print(f"  RMSE: {val_rmse:.4f}")
print(f"  MAE: {val_mae:.4f}")
print(f"\nXGBoost:")
print(f"  RMSE: {val_rmse_xgb:.4f}")
print(f"  MAE: {val_mae_xgb:.4f}")

if val_rmse < val_rmse_xgb:
    print("\n🥇 LightGBM tem melhor performance!")
    best_model = lgb_model
else:
    print("\n🥇 XGBoost tem melhor performance!")
    best_model = xgb_model

## 7. 💾 Salvamento do Modelo

In [ ]:
# Salva o melhor modelo
import os
from datetime import datetime

# Cria diretório se não existir
os.makedirs(config.MODEL_PATH, exist_ok=True)

# Nome do arquivo com timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"m5_model_{best_model.model_type}_{timestamp}.pkl"
model_filepath = os.path.join(config.MODEL_PATH, model_filename)

# Salva modelo
best_model.save_model(model_filepath)

print(f"✅ Modelo salvo: {model_filepath}")

## 8. 📋 Relatório Final

In [ ]:
# Relatório final detalhado
print("\n" + "="*60)
print("🎯 RELATÓRIO FINAL - M5 FORECASTING")
print("="*60)

print(f"\n📊 DADOS UTILIZADOS:")
print(f"  • Período: {feature_data_clean['date'].min()} a {feature_data_clean['date'].max()}")
print(f"  • Total de registros: {len(feature_data_clean):,}")
print(f"  • Itens únicos: {feature_data_clean['item_id'].nunique():,}")
print(f"  • Lojas: {feature_data_clean['store_id'].nunique()}")
print(f"  • Features criadas: {len(feature_cols)}")

print(f"\n🤖 MODELO SELECIONADO: {best_model.model_type.upper()}")
if best_model.cv_scores:
    cv_rmse_mean = np.mean([r['val_rmse'] for r in best_model.cv_scores])
    cv_rmse_std = np.std([r['val_rmse'] for r in best_model.cv_scores])
    print(f"  • RMSE CV: {cv_rmse_mean:.4f} ± {cv_rmse_std:.4f}")

print(f"\n📈 PERFORMANCE NA VALIDAÇÃO:")
if best_model.model_type == 'lightgbm':
    print(f"  • RMSE: {val_rmse:.4f}")
    print(f"  • MAE: {val_mae:.4f}")
else:
    print(f"  • RMSE: {val_rmse_xgb:.4f}")
    print(f"  • MAE: {val_mae_xgb:.4f}")

print(f"\n🎯 TOP 5 FEATURES MAIS IMPORTANTES:")
for i, (_, row) in enumerate(best_model.feature_importance.head(5).iterrows()):
    print(f"  {i+1}. {row['feature']}: {row['importance']:.0f}")

print(f"\n💾 ARQUIVOS GERADOS:")
print(f"  • Modelo: {model_filepath}")

print(f"\n🏆 CONCLUSÕES:")
print(f"  • Modelo treinado com sucesso")
print(f"  • Features de lag e rolling mostraram-se importantes")
print(f"  • Modelo pronto para previsões futuras")

print("\n" + "="*60)

## 9. 🔮 Exemplo de Previsão Futura (Demonstração)

In [ ]:
# Demonstração de como fazer previsões futuras
print("🔮 Demonstração de previsão futura...")

# Para uma implementação completa, seria necessário:
# 1. Criar features para os próximos 28 dias
# 2. Usar rolling forecast (predizer 1 dia, atualizar features, predizer próximo)
# 3. Lidar com features de lag para dias futuros

# Aqui vamos mostrar o processo com os últimos dados disponíveis
last_data = val_data.tail(1000)  # Últimos dados como exemplo
X_future, _ = best_model.prepare_training_data(last_data, feature_cols)

# Faz predição
future_predictions = best_model.predict(X_future)

print(f"\n📈 Exemplo de previsões:")
print(f"  • Número de previsões: {len(future_predictions)}")
print(f"  • Demanda média prevista: {future_predictions.mean():.2f}")
print(f"  • Demanda mínima prevista: {future_predictions.min():.2f}")
print(f"  • Demanda máxima prevista: {future_predictions.max():.2f}")

# Visualiza algumas previsões
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(future_predictions[:100], label='Previsões', marker='o', markersize=3)
plt.title('Exemplo de Previsões Futuras (Primeiros 100 pontos)')
plt.xlabel('Pontos de Previsão')
plt.ylabel('Demanda Prevista')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n✅ Demonstração concluída!")

## 📚 Próximos Passos

Para uma implementação completa em produção, considere:

1. **🔄 Rolling Forecast**: Implementar previsão rolling para 28 dias futuros
2. **🎛️ Hyperparameter Tuning**: Otimizar parâmetros com Optuna ou GridSearch
3. **🏗️ Ensemble Models**: Combinar múltiplos modelos para melhor performance
4. **📊 WRMSSE Oficial**: Implementar a métrica oficial da competição
5. **🚀 MLOps**: Automatizar pipeline de retreinamento
6. **📈 Monitoramento**: Tracking de performance em produção
7. **💾 Feature Store**: Centralizar features para reutilização
8. **🔍 Interpretabilidade**: SHAP values para explicar predições

---
**✨ Projeto M5 Forecasting - Desenvolvido com abordagem modular e escalável**